In [78]:
import pandas as pd

In [79]:
import pickle

In [80]:
import os

In [81]:
delta_headers_df = pd.read_csv('/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/airlineheaders.csv')

In [82]:
delta_headers = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
 'FlightDate', 'Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline', 
 'Origin', 'OriginCityName', 'OriginState', 'OriginStateName', 
 'Dest', 'DestCityName', 'DestState', 'DestStateName',
 'CRSDepTime', 'DepTime','DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepTimeBlk', 
 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 
 'CRSArrTime', 'ArrTime', 'ArrDelay','ArrDelayMinutes', 'ArrDel15', 'ArrTimeBlk', 
 'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 
 'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 
 'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime']

A breakdown of some of the less obvious field names. 

- CRSDEPTIME stands for Computer Reservation System (scheduled) Departure Time. 

- Depature Time is in hhmm format 
- DepDelay is in minutes
- DepDelayMinutes is absolute value of DepDelay
- DepDel15 is the delay greater than 15 minutes
- DepatureDelayGroups is Depature delay 15 minute interval group

- DepTimeBlk stands for Computer Reservation System(scheduled time block)

- CRSArrTime Computer Reservation System(scheduled) Arrival Time

- Cancelled 1 is canceled
- CancellationCode states why the flight is canceled A = Carrier, B = Weather, C = National Air System, D = Security
- Diverted 1 = diverted

- CRSElapsedTime stands for Computer Reservation System(scheduled) elapsed time

- Airtime stands for Flight time in minutes
- Flights stands for Number of flights

- Distance stands for distance between airports(miles)

- FirstDepTime stands for First gate departure time at origin airport

- TotalAddGTime stands for Total ground time away from gate

- LongestAddGTime stands for Longest time away from gate

In [83]:
download_path = '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data'

In [90]:
def chunk_df(download_path, headers, chunk_size=500000):
    
    reader = pd.read_csv(os.path.join(download_path,'airline.csv'), usecols = headers, chunksize=chunk_size, encoding = 'cp1252', low_memory = False, iterator=True)

    for chunk in reader:      
        
        try: 
            yield chunk
        
        except:
            continue

In [91]:
def chunk_year_dict(download_path, chunk_size=500000):
    
    file_path = os.path.join(download_path,'yearindex.pkl')
    
    if os.path.isfile(file_path):
        
        pass
    
    else:
        
        year_index_dict = {}
        
        gen = chunk_df(download_path,'Years', chunk_size)
           
        for index, df in enumerate(gen):
            
            try:   
                years = df['Year'].unique()
            
            except:
                
                continue
            
            for year in years:
                
                if year not in year_index_dict:
                    
                    year_index_dict[year] = []
                
                year_index_dict[year].append(index)
            
        f = open(file_path,"wb")
        pickle.dump(year_index_dict,f, pickle.HIGHEST_PROTOCOL)
        f.close()

    return file_path

In [112]:
with open(chunk_year_dict(download_path), "rb") as read_file:
        
        year_dict = pickle.load(read_file)
    
read_file.close()

In [114]:
def chunk_to_pickle(download_path, year, airline, headers, chunk_size=500000):
    
    gen = chunk_df(download_path, headers, chunk_size)
    
    with open(chunk_year_dict(download_path), "rb") as read_file:
        
        year_dict = pickle.load(read_file)
    
    read_file.close()
    
    file_path_list = []
    
    for chunk_index in year_dict[year]:
        
        file_path = os.path.join(download_path,f'{chunk_index}{airline}{year}.pickle')
        
        if os.path.isfile(file_path):
            
            file_path_list.append(file_path)
        
        else:
            
            pass
            
    if not file_path_list: 
    
        for index, df in enumerate(gen):

            if index not in year_dict[year]:

                continue

            else:

                mask = (df['Year'] == year) & (df['Reporting_Airline'] == airline)

                file_path = os.path.join(download_path,f'{index}{airline}{year}.pickle')

                with open(file_path, 'wb') as to_write:

                    pickle.dump(df[mask], to_write)
                
                to_write.close()
                
                file_path_list.append(file_path)
    
    return file_path_list

In [115]:
chunk_to_pickle(download_path, 2019, 'DL', delta_headers)

['/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/370DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/371DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/372DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/373DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/374DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/375DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/376DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/377DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/378DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/379DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/380DL2019.pickle',
 '/home/desbrium/Meti